In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn import svm
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn import tree
from sklearn.model_selection import train_test_split
import random

In [3]:
data_subset = pd.read_excel('drive/MyDrive/DBLab_Onehot/subset_data.xlsx') #확장 데이터
data_original = pd.read_excel('drive/MyDrive/DBLab_Onehot/mental_health_medical_data.xlsx') #기존 데이터

In [4]:
data = pd.concat([data_subset, data_original]) #stratify 오류 방지를 위해 각 라벨 별로 최소 2개 이상의 데이터를 가지기 위한 concat

In [5]:
# 증상의 문자열 벗기기 함수
def listing(x):
    x = eval(x)
    return x

#모델 예측 정확도를 구하기 위한 간단한 함수
def getAccuracy(prediction, target):
  count = 0
  for x, y in zip(prediction, target):
    if x == y:
      count += 1  
  return count/len(prediction) * 100

def get_clf_eval(y_test, pred):
  accuracy = accuracy_score(y_test, pred)
  f1 = f1_score(y_test, pred, average='weighted')
  return (accuracy, f1)

In [6]:
data['symptoms'] = data['symptoms'].apply(listing)

In [7]:
#레이블 인코딩(타겟 문자열 불가)
from sklearn.preprocessing import LabelEncoder
items = data['disease']
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)

print('인코딩 변환값:',labels)

인코딩 변환값: [ 0  2  2 ... 11 58  4]


In [8]:
data

,Unnamed: 0,disease,symptoms,department
0,0.0,강박 장애,"[불안, 반복적인 동작, 의심]",정신건강의학과
1,1.0,경계성 인격장애,"[의심, 약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울,...",정신건강의학과
2,2.0,경계성 인격장애,"[의심, 약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울]",정신건강의학과
3,3.0,경계성 인격장애,"[약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울]",정신건강의학과
4,4.0,경계성 인격장애,"[의심, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울]",정신건강의학과
...,...,...,...,...
59,59.0,혈관성 치매,"[언어장애, 인지장애, 감각 이상, 마비]",정신건강의학과
60,60.0,히스테리성 인격장애,[],정신건강의학과
61,61.0,레트 증후군,"[상동적 행동, 인지장애, 운동장애, 정신 지체]",정신건강의학과
62,62.0,폐소공포증,"[불안, 좁은 공간에서의 불안감]",정신건강의학과


In [9]:
X_train = np.array([x for x in data['symptoms']])
Y_train = np.array(labels)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, shuffle=True, stratify = Y_train, random_state=34) #0.8의 학습데이터, 0.2의 테스트데이터로 분리

In [11]:
x_test

array([list(['온몸이 떨림', '생리불순', '손떨림', '가슴 두근거림', '얼굴이 화끈거림', '구강건조', '권태감', '구토', '가슴 쓰림', '복부팽만감']),
       list(['얼굴이 화끈거림', '근육통', '두통']),
       list(['온몸이 떨림', '오심', '생리불순', '가슴 답답', '가슴 두근거림', '얼굴이 화끈거림', '복부팽만감', '어지러움', '저림']),
       ..., list(['오심', '얼굴이 화끈거림', '권태감', '가슴 쓰림', '복부팽만감', '두통']),
       list(['손떨림', '가슴 답답', '가슴 두근거림', '얼굴이 화끈거림', '구토', '저림']),
       list(['오심', '가슴 답답', '가슴 두근거림', '얼굴이 화끈거림', '구강건조', '근육통', '구토', '가슴 쓰림', '복부팽만감', '두통', '저림'])],
      dtype=object)

# **모델링**

In [12]:
#파라미터 설정
vocab_size = 178
embedding_dim = 8 # 임베딩 차원
max_length = 17      # 증상 최대 개수 
padding_type='post'

In [13]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

In [14]:
word_index

{'손떨림': 1,
 '가슴 두근거림': 2,
 '얼굴이 화끈거림': 3,
 '가슴 답답': 4,
 '온몸이 떨림': 5,
 '어지러움': 6,
 '두통': 7,
 '오심': 8,
 '구강건조': 9,
 '복부팽만감': 10,
 '구토': 11,
 '근육통': 12,
 '권태감': 13,
 '생리불순': 14,
 '가슴 쓰림': 15,
 '저림': 16,
 '불면증': 17,
 '환각': 18,
 '불안': 19,
 '환청': 20,
 '가슴 통증': 21,
 '기운없음': 22,
 '오한': 23,
 '발한': 24,
 '복시': 25,
 '서맥': 26,
 '숨막히는 느낌': 27,
 '고혈압': 28,
 '죽음에 대한 공포': 29,
 '저혈압': 30,
 '청색증': 31,
 '감정 변화': 32,
 '언어장애': 33,
 '초조함': 34,
 '섬망': 35,
 '혼돈': 36,
 '환시': 37,
 '수면장애': 38,
 '감각 이상': 39,
 '기억장애': 40,
 '피로감': 41,
 '실인증': 42,
 '목의 이물감': 43,
 '조급증': 44,
 '실행증': 45,
 '방향감각 상실': 46,
 '빈뇨': 47,
 '판단력장애': 48,
 '빈호흡': 49,
 '시력 감소': 50,
 '근육긴장': 51,
 '복부 불편감': 52,
 '낮은 자존감': 53,
 '의식 변화': 54,
 '대인관계 어려움': 55,
 '우울': 56,
 '망상': 57,
 '건망증': 58,
 '산만함': 59,
 '인지장애': 60,
 '수면 무호흡': 61,
 '인내심이 부족함': 62,
 '대화에 불쑥 끼어듬': 63,
 '복부 통증': 64,
 '수면 유지가 어려움': 65,
 '낮 시간대의 졸음': 66,
 '부정맥': 67,
 '수면 시작이 어려움': 68,
 '와해된 행동': 69,
 '수면 중 이상한 행동': 70,
 '호흡곤란': 71,
 '입수면기의 환각': 72,
 '수면 주기의 변화': 73,
 '흉부압박감': 74,
 '와해된 언어'

# ***테스트 데이터셋 변형***

In [ ]:
#테스트 데이터셋에 기존 증상을 제외한 일치하지 않는 증상 랜덤으로 추가
def add_offset(y_test, x_test):
  for disease, symptoms in zip(y_test, x_test):
    disease_name =  encoder.classes_[disease]
    ind = -1
    for index, i in enumerate(data_original['disease']):
      if disease_name == i:
        ind = index
    flag = True
    while flag:
      rand_ind = randint(1, 177)
      for k, symp in enumerate(word_index):
        if k+1 == rand_ind:
          if symp in data_original.iloc[ind]['symptoms']:
            break
          else:
            symptoms.append(symp)
            flag = False
            break

In [ ]:
add_offset(y_test, x_test)
x_test

array([list(['온몸이 떨림', '생리불순', '손떨림', '가슴 두근거림', '얼굴이 화끈거림', '구강건조', '권태감', '구토', '가슴 쓰림', '복부팽만감', '지남력 장애']),
       list(['얼굴이 화끈거림', '근육통', '두통', '자제력 상실 두려움']),
       list(['온몸이 떨림', '오심', '생리불순', '가슴 답답', '가슴 두근거림', '얼굴이 화끈거림', '복부팽만감', '어지러움', '저림', '와해된 행동']),
       ...,
       list(['오심', '얼굴이 화끈거림', '권태감', '가슴 쓰림', '복부팽만감', '두통', '청색증']),
       list(['손떨림', '가슴 답답', '가슴 두근거림', '얼굴이 화끈거림', '구토', '저림', '병원 쇼핑']),
       list(['오심', '가슴 답답', '가슴 두근거림', '얼굴이 화끈거림', '구강건조', '근육통', '구토', '가슴 쓰림', '복부팽만감', '두통', '저림', '체중증가'])],
      dtype=object)

In [15]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(x_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [ ]:
test_padded

array([[ 5, 14,  1, ...,  0,  0,  0],
       [ 3, 12,  7, ...,  0,  0,  0],
       [ 5,  8, 14, ...,  0,  0,  0],
       ...,
       [ 8,  3, 13, ...,  0,  0,  0],
       [ 1,  4,  2, ...,  0,  0,  0],
       [ 8,  4,  2, ...,  0,  0,  0]], dtype=int32)

# ***NLP 모델***

In [22]:
Models = [0,0,0,0,0]

In [16]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(64, activation='softmax')
])

In [17]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 8)             1424      
                                                                 
 global_average_pooling1d (G  (None, 8)                0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                216       
                                                                 
 dense_1 (Dense)             (None, 64)                1600      
                                                                 
Total params: 3,240
Trainable params: 3,240
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
# fit model
num_epochs = 20
#history = model.fit(train_padded, y_train, 
                    #epochs=1000, verbose=2)
history = model.fit(train_padded, y_train, 
                     epochs=num_epochs, verbose=2, 
                     validation_split=0.2)

Epoch 1/20
1577/1577 - 3s - loss: 0.6598 - accuracy: 0.8929 - val_loss: 0.1742 - val_accuracy: 0.9510 - 3s/epoch - 2ms/step
Epoch 2/20
1577/1577 - 2s - loss: 0.1006 - accuracy: 0.9760 - val_loss: 0.0737 - val_accuracy: 0.9827 - 2s/epoch - 1ms/step
Epoch 3/20
1577/1577 - 2s - loss: 0.0552 - accuracy: 0.9859 - val_loss: 0.0559 - val_accuracy: 0.9842 - 2s/epoch - 1ms/step
Epoch 4/20
1577/1577 - 2s - loss: 0.0408 - accuracy: 0.9890 - val_loss: 0.0429 - val_accuracy: 0.9895 - 2s/epoch - 2ms/step
Epoch 5/20
1577/1577 - 3s - loss: 0.0302 - accuracy: 0.9924 - val_loss: 0.0342 - val_accuracy: 0.9922 - 3s/epoch - 2ms/step
Epoch 6/20
1577/1577 - 2s - loss: 0.0240 - accuracy: 0.9936 - val_loss: 0.0286 - val_accuracy: 0.9937 - 2s/epoch - 1ms/step
Epoch 7/20
1577/1577 - 4s - loss: 0.0201 - accuracy: 0.9947 - val_loss: 0.0251 - val_accuracy: 0.9944 - 4s/epoch - 3ms/step
Epoch 8/20
1577/1577 - 3s - loss: 0.0173 - accuracy: 0.9954 - val_loss: 0.0223 - val_accuracy: 0.9952 - 3s/epoch - 2ms/step
Epoch 9/

In [19]:
# 학습된 모델로 테스트 데이터 예측
y_prob = model.predict(test_padded, verbose=2) 
predicted = y_prob.argmax(axis=-1)

encoder.classes_[predicted]

493/493 - 0s - 364ms/epoch - 739us/step


array(['신체형 장애', '신체형 장애', '신체형 장애', ..., '신체형 장애', '신체형 장애', '신체형 장애'],
      dtype=object)

In [ ]:
acc_mlp, fl_mlp = get_clf_eval(y_test, predicted)

print("Accuracy: " + str(acc_mlp) + ", " + "F1 Score: " + str(fl_mlp))

Accuracy: 0.9932762448461783, F1 Score: 0.9927891898141622


In [23]:
Models[0] = model

# ***SVM***


In [ ]:
#SupportVector 모델 학습
SVM = svm.SVC(C=1.0, kernel='poly', degree=3, gamma='auto', verbose=True, probability=True)
SVM.fit(train_padded, y_train)

In [ ]:
predictions_SVM = SVM.predict(test_padded)
predictions_SVM

encoder.classes_[predictions_SVM]

In [ ]:
acc_svm, fl_svm = get_clf_eval(y_test, predictions_SVM)

print("Accuracy: " + str(acc_svm) + ", " + "F1 Score: " + str(fl_svm))

In [ ]:
Models[1] = SVM

# ***Naive Bayes***

In [24]:
#Naive bayes 모델 학습
Naive = naive_bayes.MultinomialNB()
Naive.fit(train_padded, y_train)

MultinomialNB()

In [25]:
predictions_NB = Naive.predict(test_padded)

encoder.classes_[predictions_NB]

array(['신체형 장애', '강박 장애', '신체형 장애', ..., '금단 현상', '신체형 장애', '신체형 장애'],
      dtype=object)

In [26]:
acc_nb, fl_nb = get_clf_eval(y_test, predictions_NB)

print("Accuracy: " + str(acc_nb) + ", " + "F1 Score: " + str(fl_nb))

Accuracy: 0.5888360291785601, F1 Score: 0.6885924032139362


In [27]:
Models[2] = Naive

# ***Decision Tree***

In [43]:
#DecisionTree 모델 학습
DTC = tree.DecisionTreeClassifier(min_samples_leaf=17)
DTC.fit(train_padded, y_train)

DecisionTreeClassifier(min_samples_leaf=17)

In [44]:
predictions_DTC = DTC.predict(test_padded)

encoder.classes_[predictions_DTC]

array(['신체형 장애', '신체형 장애', '신체형 장애', ..., '신체형 장애', '신체형 장애', '신체형 장애'],
      dtype=object)

In [45]:
acc_dtc, fl_dtc = get_clf_eval(y_test, predictions_DTC)

print("Accuracy: " + str(acc_dtc) + ", " + "F1 Score: " + str(fl_dtc))

Accuracy: 0.9941642879797019, F1 Score: 0.9935858680906577


In [46]:
Models[3] = DTC

---
# 여기서부터 추가 



더이상 입력할 증상이 없으면 종료
90%는 넘겨보자

안되면 어떤 증상을 물어보지?

현재 입력한 증상이 있는 질병중 가장 많이 출현하는 증상을 물어보자 예를 들면

"당신은 (해당 증상)을 가졌을 확률이 높습니다."

전제
1. 환자가 거짓된 증상을 입력하지 않는다
2. 오타가 나지 않는다 >> 뛰어쓰기같은건 replace(" ", "")로 처음부터 지우고 나머지는 다시 하라고 하자

In [32]:
def find_mode(pat_word_index):#입력받은 증상 배열
  ##어떤 질병에 포함된 증상인지
  sample_train = []#증상의 index
  torf = True
  for i in range(len(data['symptoms'])):#전체 증상
    for j in range(len(pat_word_index)):#입력받은 증상
      torf = pat_word_index[j] in data['symptoms'][i]
      if(not torf):break
    if(torf): sample_train.append(i)
    ##모든 증상 시리즈를 다 돌면서 한 시리즈에 환자의 증상이 다 들어가 있으면 torf는 계속 true, 끝까지 true면 해당 질병을 append

  ld_symp = []#주어진 증상 배열을 포함하는 질병들의 증상 목록
  for i in data['symptoms'][sample_train]:
    for j in i:
      ld_symp.append(j)

  max = -1
  max_arr = []
  chk_symp = pat_word_index.copy()
  for i in ld_symp:#어떤 증상 i를 확인해볼까?
    count = 0
    if(i not in chk_symp):
      for j in ld_symp:#이 증상이 몇 개 있는지 배열을 돌아볼까?
        if(i == j): count += 1#i 증상이 j로 나오면 1회 추가
      
      if(count > max):#새로운 최빈값
        max_arr = [i]
        max = count

      elif(count == max):#이것도 최빈값
        max_arr.append(i)

    chk_symp.append(i)
  return max_arr

In [ ]:
model_use = 3
#증상 목록 출력
# for i in word_index.keys():
#   print(i)

#증상 수 초과시 종료
count = 1

#첫 증상 입력

##증상 입력받을 배열
pat_word_index = []
pat_symp = input("\n위 증상중 해당하는 증상을 하나씩 입력해주세요 (없으면 엔터) : ")
pat_word_index.append(pat_symp)

pat_word_array = np.array(pd.Series([pat_word_index]))

##데이터를 sequence로 변환해주고 padding 해줍니다.
pat_sequences = tokenizer.texts_to_sequences(pat_word_array.tolist())
pat_padded = pad_sequences(pat_sequences, padding=padding_type, maxlen=max_length)

##확률 계산
if model_use == 0:
  y_prob = Models[model_use].predict(pat_padded)
else:
  y_prob = Models[model_use].predict_proba(pat_padded)
predicted = y_prob.argmax(axis=-1)

print(y_prob)

#두번째 이상의 증상 입력
del_token = "[']"
asked = [] #물어봤던 증상 배열

while (y_prob[0][predicted[0]] < 0.9) and (pat_symp !='') and (count <= vocab_size):#1. 확률이 90% 미만, 빈칸이 아님, 최대치만큼 안돌음
  target_symptoms = data_subset['symptoms'][data_subset['disease'].ne(encoder.classes_[predicted][0]).idxmin()]
  
  for t in del_token:
    target_symptoms = target_symptoms.replace(t, "")
  target_symptoms = target_symptoms.split(',')

  new = []

  for t in target_symptoms:
    if len(t) > 0 and t[0] == " ":
      t = t[1:]
    new.append(t)
  
  for t in new:
    if t in pat_word_index:
      new.remove(t)

  count = 0

  while len(new) > 0:
    count += 1
    rand_ind = random.randrange(0, len(new))

    next_symp = new[rand_ind]
    if next_symp in asked: #물어봤던 증상일 경우 넘어감
      del new[rand_ind]
      continue

    yn = input("\n" + str(count) + ". " + next_symp + " 증상이 있으십니까? (y/n): ")

    if yn == 'y':
      pat_symp = next_symp
      break;
    else:
      asked.append(next_symp)
      del new[rand_ind]

  pat_word_index.append(pat_symp)

  pat_word_array = np.array(pd.Series([pat_word_index]))

  ##데이터를 sequence로 변환해주고 padding 해줍니다.
  pat_sequences = tokenizer.texts_to_sequences(pat_word_array.tolist())
  pat_padded = pad_sequences(pat_sequences, padding=padding_type, maxlen=max_length)

  ##확률 계산
  y_prob = model.predict(pat_padded, verbose=0)
  predicted = y_prob.argmax(axis=-1)

  count += 1

pat_word_index = pat_word_index[:-1]
print("예측이 완료되었습니다.")


위 증상중 해당하는 증상을 하나씩 입력해주세요 (없으면 엔터) : 망상
[[0.         0.         0.         0.         0.         0.07692308
  0.         0.         0.07692308 0.         0.07692308 0.
  0.07692308 0.07692308 0.07692308 0.         0.         0.07692308
  0.         0.         0.07692308 0.         0.         0.
  0.         0.07692308 0.         0.         0.         0.
  0.         0.         0.         0.         0.07692308 0.
  0.         0.         0.07692308 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.07692308 0.         0.         0.         0.         0.
  0.         0.         0.07692308 0.         0.         0.
  0.         0.         0.         0.        ]]

1. 대인관계 어려움 증상이 있으십니까? (y/n): n

2. 불안 증상이 있으십니까? (y/n): n

1. 식욕부진 증상이 있으십니까? (y/n): n

2. 오후 우울감 완화 증상이 있으십니까? (y/n): n

3. 환각 증상이 있으십니까? (y/n): n

4. 수면장애 증상이 있으십니까? (y/n): n

5. 자살의 위험 증상이 있으십니까? (y/n): n

6. 삶에 대한 흥미 상실 증상이 있으십니까? (y/n): n

1. 와해된 행동 증상이 있으십니까? (y/n): n

2. 피해망상 증상이 있으십니까

In [77]:
#결과 출력

##확률이 90%가 넘어서 멈춘 경우
if(y_prob[0][predicted[0]] >0.9):
  print("당신의 증상은 "+str(y_prob[0][predicted[0]]*100)+"%의 확률로 "+encoder.classes_[predicted][0]+" 입니다.")

##더이상 입력을 안한 경우, 즉 입력을 완료한 경우
else:
  print("현재 입력해주신 증상으로는 "+str(y_prob[0][predicted[0]]*100)+"%의 확률로 "+encoder.classes_[predicted][0]+" 입니다.")

당신의 증상은 92.79376268386841%의 확률로 경계성 인격장애 입니다.


해야 할 내용

---

* 알코올성 치매, 히스테리성 인격장애는 증상이 없고, 증상이 없는 경우 확률은 30%대가 최대라 판별이 불가능한데 어떻게 해결할까 >> 일정 확률 이상의 질병들을 모두 출력할까?